This Query is created by user sohamjcpp

In [2]:
%% sql
```
WITH Premia_Customer AS (
  SELECT cust_code AS intermediary_code, cust_name AS intermediary_name, 
         cust_code AS fufiller_code
  FROM Premia_Pcom_customer
  WHERE SUBSTRING(cust_code, 1, 2) = (SELECT dim_intermediary_type FROM policy_master_agg)
),
Bancs_Party AS (
  SELECT party_code AS intermediary_code, party_name AS intermediary_name
  FROM Bancs_T_RPDB_POL_PARTY
),
Derived AS (
  SELECT CASE WHEN CURRENT_DATE - aml_acknowledged_timestamp >= 365 THEN 'Yes' ELSE 'No' END AS due_aml_training,
         CURRENT_DATE AS reported_date,
         CURRENT_TIMESTAMP AS load_timestamp
)
SELECT PCUST.intermediary_code, PCUST.intermediary_name, 
       PCUST.intermediary_type, PCUST.agent_mobile_no, PCUST.agent_email_id,
       VO.is_aml_acknowledged, VO.aml_acknowledged_timestamp,
       DER.due_aml_training, DER.reported_date, DER.load_timestamp,
       PMRD.fufiller_code, PMRD.fulfiller_name, PMRD.office_code,
       PMRD.office_name, PMRD.zone_name, PMRD.mkt_zone_name
FROM Premia_Customer AS PCUST
LEFT JOIN Bancs_Party AS BPARTY ON PCUST.intermediary_code = BPARTY.intermediary_code
LEFT JOIN vo_login AS VO ON PCUST.intermediary_code = VO.intermediary_code
LEFT JOIN Derived AS DER ON VO.aml_acknowledged_timestamp = DER.aml_acknowledged_timestamp
LEFT JOIN agg_policy_master_report_daily AS PMRD ON PCUST.intermediary_code = PMRD.intermediary_code
ORDER BY PCUST.intermediary_code;
```